# KIT DE EMERGENCIA PARA EL EXAMEN DE spaCy

In [1]:
# 1. Importaciones necesarias
import spacy
from spacy.tokens import Span, Doc
from spacy.language import Language
import re
from spacy import displacy

In [2]:
# 2. Definir extensiones personalizadas
Span.set_extension("numero", default=None, force=True)
Span.set_extension("titulo", default=None, force=True)
Span.set_extension("longitud", getter=lambda span: len(span), force=True)
Doc.set_extension("capitulos", default=[], force=True)

# 3. Crear el componente personalizado para fragmentar capítulos
@Language.component("fragmentador_capitulos")
def fragmentador_capitulos(doc):
    patron = re.compile(r"\n+(\d+)\s*\n+([^\n]+)")
    coincidencias = list(patron.finditer(doc.text))
    capitulos = []
    for i, match in enumerate(coincidencias):
        numero = int(match.group(1))
        titulo = match.group(2).strip()
        start_char = match.end()
        end_char = coincidencias[i + 1].start() if i + 1 < len(coincidencias) else len(doc.text)
        span = doc.char_span(start_char, end_char, alignment_mode="contract")
        if span is None:
            continue
        span._.numero = numero
        span._.titulo = titulo
        capitulos.append(span)
    doc._.capitulos = capitulos
    return doc

In [12]:
# ! python -m spacy download en_core_web_md

     ---------------------------------------- 0.0/33.5 MB ? eta -:--:--
     ------ --------------------------------- 5.5/33.5 MB 41.9 MB/s eta 0:00:01
     -------------- ------------------------ 12.8/33.5 MB 40.3 MB/s eta 0:00:01
     ------------------- ------------------- 17.0/33.5 MB 32.6 MB/s eta 0:00:01
     ------------------------- ------------- 21.8/33.5 MB 29.3 MB/s eta 0:00:01
     ------------------------------- ------- 27.3/33.5 MB 29.3 MB/s eta 0:00:01
     ------------------------------------- - 32.2/33.5 MB 28.1 MB/s eta 0:00:01
     --------------------------------------- 33.5/33.5 MB 26.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# 4. Cargar modelo de idioma
nlp = spacy.load("es_core_news_md")  # Usamos un pipeline vacío en español
nlp.add_pipe("fragmentador_capitulos")  # Añadimos el componente personalizado

<function __main__.fragmentador_capitulos(doc)>

# Cargar el texto del libro (puedes cambiar el nombre del archivo)

In [4]:
# 5. Cargar el texto del libro (puedes cambiar el nombre del archivo)
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 1 - La Piedra Filosofal.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto = "".join(lineas[37:])  # índice 37 = línea 38 (empieza desde 0)

print(texto[:1000])  # Muestra los primeros 1000 caracteres para verificar






1
    
El niño que vivió


El señor y la señora Dursley, que vivían en el número 4 de Privet Drive, estaban orgullosos de decir que eran muy normales, afortunadamente. Eran las últimas personas que se esperaría encontrar relacionadas con algo extraño o misterioso, porque no estaban para tales tonterías.
El señor Dursley era el director de una empresa llamada Grunnings, que fabricaba taladros. Era un hombre corpulento y rollizo, casi sin cuello, aunque con un bigote inmenso. La señora Dursley era delgada, rubia y tenía un cuello casi el doble de largo de lo habitual, lo que le resultaba muy útil, ya que pasaba la mayor parte del tiempo estirándolo por encima de la valla de los jardines para espiar a sus vecinos. Los Dursley tenían un hijo pequeño llamado Dudley, y para ellos no había un niño mejor que él.
Los Dursley tenían todo lo que querían, pero también tenían un secreto, y su mayor temor era que lo descubriesen: no habrían soportado que se supiera lo de los Potter.
La señora Pot

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
C:\Users\jorge\AppData\Local\Temp\ipykernel_18420\3145471462.py:3: SyntaxWarning: invalid escape sequence '\l'
  ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 1 - La Piedra Filosofal.txt"


In [ ]:

ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 2 - La Cámara Secreta.txt"

with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto = "".join(lineas[35:])  

print(texto[:1000])  


In [ ]:
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto = "".join(lineas[37:])  # índice 37 = línea 38 (empieza desde 0)

In [16]:
# 6. Procesar el documento
doc = nlp(texto)

#  FUNCIONES RAPIDAS DE UTILIDAD

In [33]:
def get_capitulo_por_numero(doc, numero):
    """Devuelve el capítulo cuyo número coincide."""
    for capitulo in doc._.capitulos:
        if capitulo._.numero == numero:
            return capitulo
    return None

def get_capitulo_por_titulo(doc, titulo):
    """Devuelve el capítulo cuyo título contiene el texto indicado."""
    for capitulo in doc._.capitulos:
        if titulo.lower() in capitulo._.titulo.lower():
            return capitulo
    return None

from spacy import displacy
from IPython.display import display, HTML

def visualizar_dependencias(span):
    """Muestra las dependencias de un span o fragmento."""
    html = displacy.render(span.as_doc(), style="dep", jupyter=False)
    display(HTML(html))



In [18]:
# Obtener un capítulo por número
capitulo1 = get_capitulo_por_numero(doc, 1)
print(capitulo1._.titulo)

El niño que vivió


In [34]:
# Visualizar dependencias del primer párrafo del capítulo 1
primer_parrafo = list(capitulo1.sents)[0]  # Primer frase (sentence)
visualizar_dependencias(primer_parrafo)

In [30]:

# Acceder a los datos del capítulo
print("Longitud del capítulo:", capitulo1._.longitud)
print("Texto del capítulo:", capitulo1.text[:500])  # Primeros 500 caracteres

Longitud del capítulo: 5602
Texto del capítulo: 


El señor y la señora Dursley, que vivían en el número 4 de Privet Drive, estaban orgullosos de decir que eran muy normales, afortunadamente. Eran las últimas personas que se esperaría encontrar relacionadas con algo extraño o misterioso, porque no estaban para tales tonterías.
El señor Dursley era el director de una empresa llamada Grunnings, que fabricaba taladros. Era un hombre corpulento y rollizo, casi sin cuello, aunque con un bigote inmenso. La señora Dursley era delgada, rubia y tenía 
